In [8]:
import json
from collections import defaultdict

with open("expansion_recipes.json") as f:
    recipes_data = json.load(f)

# Create a lookup table for recipes by output item
recipes_by_output = defaultdict(list)
for recipe in recipes_data:
    for item, amount in recipe['Deltas'].items():
        if amount > 0:  # Only consider outputs for the lookup
            recipes_by_output[item].append(recipe)

productivities = defaultdict(lambda: 1, {"Foundry": 1.9, "Chemical Plant": 1.3, "Assembling Machine": 1.4, "Electromagnetic Plant": 1.9, "Rocket Silo": 1.4})
for recipe in recipes_data:
    for item, amount in recipe['Deltas'].items():
        if amount > 0:
            recipe['Deltas'][item] *= productivities[recipe['Building']]

def calculate_requirements(target_item, quantity, minimal_items, depth=0, max_depth=15):
    # Terminate if recursion depth is too high
    if depth > max_depth:
        return None

    requirements = defaultdict(float)
    
    # Check if the target item is a minimal item or time
    if target_item in minimal_items or target_item == "Time":
        requirements[target_item] = quantity
        return requirements

    # Try each recipe that produces the target item
    if target_item not in recipes_by_output:
        return None  # No recipe found for the target item
    
    for recipe in recipes_by_output[target_item]:
        output_qty = recipe["Deltas"][target_item]
        multiplier = quantity / output_qty
        temp_requirements = defaultdict(float)

        for ingredient, amount in recipe["Deltas"].items():
            if ingredient == target_item:
                continue
            if amount > 0:
                continue
            needed_qty = multiplier * -amount if amount < 0 else multiplier * amount
            sub_requirements = calculate_requirements(ingredient, needed_qty, minimal_items, depth + 1, max_depth)
            
            if sub_requirements is None:
                break  # This recipe path is invalid; try another recipe
            else:
                # Accumulate requirements from the valid sub-path
                for item, qty in sub_requirements.items():
                    temp_requirements[item] += qty
        else:
            # If the loop didn't break, the recipe path is valid
            for item, qty in temp_requirements.items():
                requirements[item] += qty
            return requirements

    # If no recipe path worked, return None
    return None

# Example usage
minimal_items = ["Stone", "Calcite", "Lava", "Tungsten ore", "Sulfuric acid", "Coal"]  # Define your minimal items here
target_item = "Rocket"  # Define your target item
quantity_needed = 1.0  # Define how much of the target item you want to produce

requirements = calculate_requirements(target_item, quantity_needed, minimal_items)
if requirements:
    print("Minimal item requirements:\n", dict(requirements))
else:
    print("No valid way to produce the target item.")



Minimal item requirements:
 {'Time': 2918.0520570406934, 'Calcite': 18.15071928389262, 'Lava': 8759.37166832644, 'Coal': 1694.505762114054, 'Sulfuric acid': 725.9609096457523}
